# Semantic Routing

RedisVL provides a `SemanticRouter` interface to utilize Redis' built-in search & aggregation in order to perform
KNN-style classification over a set of `Route` references to determine the best match.

This notebook will go over how to use Redis as a Semantic Router for your Java applications

First, we will set up our dependencies.

In [1]:
// Load Maven dependencies
%maven redis.clients:jedis:5.2.0
%maven org.slf4j:slf4j-nop:2.0.16
%maven com.microsoft.onnxruntime:onnxruntime:1.16.3
%maven com.squareup.okhttp3:okhttp:4.12.0
%maven com.fasterxml.jackson.core:jackson-databind:2.18.2
%maven com.fasterxml.jackson.dataformat:jackson-dataformat-yaml:2.18.2
%maven com.fasterxml.jackson.datatype:jackson-datatype-jsr310:2.18.2

// Note: RedisVL JAR must be in classpath (loaded automatically by Docker container)

// Import RedisVL classes
import com.redis.vl.extensions.router.*;
import com.redis.vl.utils.vectorize.*;

// Import Redis client
import redis.clients.jedis.UnifiedJedis;
import redis.clients.jedis.HostAndPort;

// Import Java standard libraries
import java.util.*;

System.out.println("Dependencies loaded");

Dependencies loaded


## Define the Routes

Below we define 3 different routes. One for `technology`, one for `sports`, and
another for `entertainment`. Now for this example, the goal here is
surely topic "classification". But you can create routes and references for
almost anything.

Each route has a set of references that cover the "semantic surface area" of the
route. The incoming query from a user needs to be semantically similar to one or
more of the references in order to "match" on the route.

Additionally, each route has a `distanceThreshold` which determines the maximum distance between the query and the reference for the query to be routed to the route. This value is unique to each route.

In [2]:
// Define routes for the semantic router
Route technology = Route.builder()
    .name("technology")
    .references(List.of(
        "what are the latest advancements in AI?",
        "tell me about the newest gadgets",
        "what's trending in tech?"
    ))
    .metadata(Map.of("category", "tech", "priority", 1))
    .distanceThreshold(0.71)
    .build();

Route sports = Route.builder()
    .name("sports")
    .references(List.of(
        "who won the game last night?",
        "tell me about the upcoming sports events",
        "what's the latest in the world of sports?",
        "sports",
        "basketball and football"
    ))
    .metadata(Map.of("category", "sports", "priority", 2))
    .distanceThreshold(0.72)
    .build();

Route entertainment = Route.builder()
    .name("entertainment")
    .references(List.of(
        "what are the top movies right now?",
        "who won the best actor award?",
        "what's new in the entertainment industry?"
    ))
    .metadata(Map.of("category", "entertainment", "priority", 3))
    .distanceThreshold(0.7)
    .build();

List<Route> routes = List.of(technology, sports, entertainment);
System.out.println("Defined " + routes.size() + " routes");

Defined 3 routes


## Initialize the SemanticRouter

`SemanticRouter` will automatically create an index within Redis upon initialization for the route references. We'll use the `SentenceTransformersVectorizer` which downloads and runs HuggingFace models locally using ONNX Runtime.

On first use, it will download the `sentence-transformers/all-mpnet-base-v2` model and cache it locally in `~/.cache/redisvl4j/models/`. Subsequent runs will use the cached model for fast initialization.

In [3]:
// Connect to Redis
UnifiedJedis jedis = new UnifiedJedis(new HostAndPort("redis-stack", 6379));

// Create vectorizer using SentenceTransformersVectorizer to download and run model locally
BaseVectorizer vectorizer = new SentenceTransformersVectorizer("sentence-transformers/all-mpnet-base-v2");

System.out.println("Initializing vectorizer with sentence-transformers/all-mpnet-base-v2 model...");
System.out.println("Model dimensions: " + vectorizer.getDimensions());

// Initialize the SemanticRouter
SemanticRouter router = SemanticRouter.builder()
    .name("topic-router")
    .vectorizer(vectorizer)
    .routes(routes)
    .jedis(jedis)
    .overwrite(true) // Blow away any other routing index with this name
    .build();

System.out.println("SemanticRouter initialized with index: " + router.getName());

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Initializing vectorizer with sentence-transformers/all-mpnet-base-v2 model...
Model dimensions: 768
SemanticRouter initialized with index: topic-router


In [4]:
// Look at the index specification created for the semantic router
Map<String, Object> indexInfo = router.getIndex().info();
System.out.println("Index name: " + indexInfo.get("index_name"));
System.out.println("Storage type: HASH");
System.out.println("Number of documents: " + indexInfo.get("num_docs"));

Index name: topic-router
Storage type: HASH
Number of documents: 11


## Simple routing

In [5]:
// Query the router with a statement
RouteMatch routeMatch = router.route("Can you tell me about the latest in artificial intelligence?");
System.out.println("Route matched: " + routeMatch.getName());
System.out.println("Distance: " + routeMatch.getDistance());

Route matched: technology
Distance: 0.41914610068


In [6]:
// Query the router with a statement and return a miss
RouteMatch noMatch = router.route("are aliens real?");
System.out.println("Route matched: " + noMatch.getName());
System.out.println("Distance: " + noMatch.getDistance());

Route matched: null
Distance: null


We can also route a statement to many routes and order them by distance:

In [7]:
// Perform multi-class classification with routeMany() -- toggle the maxK
List<RouteMatch> routeMatches = router.routeMany("How is AI used in basketball?", 3, null, null);
for (RouteMatch match : routeMatches) {
    System.out.println("RouteMatch(name=" + match.getName() + ", distance=" + match.getDistance() + ")");
}

RouteMatch(name=technology, distance=0.556493997574)
RouteMatch(name=sports, distance=0.671060125033)


In [8]:
// Toggle the aggregation method -- note the different distances in the result
List<RouteMatch> minMatches = router.routeMany(
    "How is AI used in basketball?", 
    3, // maxK
    null, // vector (will be computed from text)
    DistanceAggregationMethod.MIN
);

for (RouteMatch match : minMatches) {
    System.out.println("RouteMatch(name=" + match.getName() + ", distance=" + match.getDistance() + ")");
}

RouteMatch(name=technology, distance=0.556493997574)
RouteMatch(name=sports, distance=0.62926441431)


Note the different route match distances. This is because we used the `min` aggregation method instead of the default `avg` approach.

## Update the routing config

In [9]:
router.updateRoutingConfig(
    RoutingConfig.builder()
        .aggregationMethod(DistanceAggregationMethod.MIN)
        .maxK(3)
        .build()
);

System.out.println("Updated routing config:");
System.out.println("  Aggregation method: " + router.getRoutingConfig().getAggregationMethod());
System.out.println("  Max K: " + router.getRoutingConfig().getMaxK());

Updated routing config:
  Aggregation method: MIN
  Max K: 3


In [10]:
List<RouteMatch> matches = router.routeMany("Lebron James");
for (RouteMatch match : matches) {
    System.out.println("RouteMatch(name=" + match.getName() + ", distance=" + match.getDistance() + ")");
}

RouteMatch(name=sports, distance=0.663254201412)


## Router serialization

In [11]:
// Serialize router configuration to a dictionary (Map)
Map<String, Object> routerDict = router.toDict();

System.out.println("Router configuration:");
System.out.println("  Name: " + routerDict.get("name"));
System.out.println("  Number of routes: " + ((List<?>) routerDict.get("routes")).size());
System.out.println("  Vectorizer model: " + ((Map<?, ?>) routerDict.get("vectorizer")).get("model"));

// Show routing config
Map<String, Object> config = (Map<String, Object>) routerDict.get("routing_config");
System.out.println("  Routing config:");
System.out.println("    max_k: " + config.get("max_k"));
System.out.println("    aggregation_method: " + config.get("aggregation_method"));

// Show route details with metadata
List<Map<String, Object>> routes = (List<Map<String, Object>>) routerDict.get("routes");
System.out.println("\nRoute details:");
for (Map<String, Object> route : routes) {
    System.out.println("  Route: " + route.get("name"));
    System.out.println("    Distance threshold: " + route.get("distance_threshold"));
    System.out.println("    Metadata: " + route.get("metadata"));
    System.out.println("    References: " + ((List<?>) route.get("references")).size());
}

Router configuration:
  Name: topic-router
  Number of routes: 3
  Vectorizer model: sentence-transformers/all-mpnet-base-v2
  Routing config:
    max_k: 3
    aggregation_method: min

Route details:
  Route: technology
    Distance threshold: 0.71
    Metadata: {category=tech, priority=1}
    References: 3
  Route: sports
    Distance threshold: 0.72
    Metadata: {category=sports, priority=2}
    References: 5
  Route: entertainment
    Distance threshold: 0.7
    Metadata: {category=entertainment, priority=3}
    References: 3


In [12]:
// Verify router configuration is persisted to Redis as JSON
// The router config is stored at key "{name}:route_config"
String configKey = router.getName() + ":route_config";
Object configJson = jedis.jsonGet(configKey);

System.out.println("Router config stored in Redis at key: " + configKey);
System.out.println("Config exists: " + (configJson != null));

if (configJson != null) {
    System.out.println("\nStored JSON includes:");
    System.out.println("  - Router name and routes with metadata");
    System.out.println("  - Distance thresholds for each route");
    System.out.println("  - Vectorizer configuration");
    System.out.println("  - Routing config (max_k, aggregation_method)");
    System.out.println("\nThis allows the router state to be fully restored from Redis");
}

Router config stored in Redis at key: topic-router:route_config
Config exists: true

Stored JSON includes:
  - Router name and routes with metadata
  - Distance thresholds for each route
  - Vectorizer configuration
  - Routing config (max_k, aggregation_method)

This allows the router state to be fully restored from Redis


## Route Reference Management

You can dynamically add, retrieve, and delete route references after the router is created.

### Add Route References

Add new references to an existing route:

In [13]:
// Add new references to the technology route
List<String> newRefs = List.of("latest AI trends", "new tech gadgets");
List<String> addedKeys = router.addRouteReferences("technology", newRefs);

System.out.println("Added " + addedKeys.size() + " references to technology route");
for (String key : addedKeys) {
    System.out.println("  Key: " + key);
}

// Verify the route now has additional references
Route techRoute = router.get("technology");
System.out.println("\nTechnology route now has " + techRoute.getReferences().size() + " references");

Added 2 references to technology route
  Key: technology:f243fb2d073774e8
  Key: technology:7e4bca5853c1c329

Technology route now has 5 references


### Get Route References

Retrieve references by route name or reference ID:

In [14]:
// Get all references for the technology route
List<Map<String, Object>> refs = router.getRouteReferences("technology", null, null);

System.out.println("Found " + refs.size() + " references for technology route:");
for (Map<String, Object> ref : refs) {
    System.out.println("  Reference: " + ref.get("reference"));
    System.out.println("  Reference ID: " + ref.get("reference_id"));
    System.out.println("  Route: " + ref.get("route_name"));
    System.out.println();
}

Found 5 references for technology route:
  Reference: what's trending in tech?
  Reference ID: 149a9c9919c58534
  Route: technology

  Reference: latest AI trends
  Reference ID: f243fb2d073774e8
  Route: technology

  Reference: tell me about the newest gadgets
  Reference ID: 85cc73a1437df27c
  Route: technology

  Reference: what are the latest advancements in AI?
  Reference ID: 851f51cce5a9ccfb
  Route: technology

  Reference: new tech gadgets
  Reference ID: 7e4bca5853c1c329
  Route: technology



In [15]:
// Get a specific reference by ID
String firstRefId = (String) refs.get(0).get("reference_id");
List<Map<String, Object>> specificRef = router.getRouteReferences(null, List.of(firstRefId), null);

System.out.println("Retrieved reference by ID:");
System.out.println("  Reference: " + specificRef.get(0).get("reference"));
System.out.println("  Reference ID: " + specificRef.get(0).get("reference_id"));

Retrieved reference by ID:
  Reference: what's trending in tech?
  Reference ID: 149a9c9919c58534


### Delete Route References

Delete references by route name or reference ID:

In [16]:
// Delete all references for the sports route
int deletedCount = router.deleteRouteReferences("sports", null, null);

System.out.println("Deleted " + deletedCount + " references from sports route");

// Verify the route now has no references
Route sportsRoute = router.get("sports");
System.out.println("Sports route now has " + sportsRoute.getReferences().size() + " references");

Deleted 5 references from sports route
Sports route now has 0 references


In [17]:
// Delete a specific reference by ID from technology route
List<Map<String, Object>> techRefs = router.getRouteReferences("technology", null, null);
String refIdToDelete = (String) techRefs.get(0).get("reference_id");

int deleted = router.deleteRouteReferences(null, List.of(refIdToDelete), null);

System.out.println("Deleted " + deleted + " reference by ID");

// Verify the deletion
Route techRouteAfter = router.get("technology");
System.out.println("Technology route now has " + techRouteAfter.getReferences().size() + " references");

Deleted 1 reference by ID
Technology route now has 4 references


## Clean up the router

When you're done, you can clear the router's data or delete the index entirely.

In [18]:
// Use clear() to flush all routes from the index
router.clear();
System.out.println("Router cleared");

Router cleared


In [19]:
// Use delete() to clear the index and remove it completely
router.delete();
System.out.println("Router deleted");
System.out.println("Index exists: " + router.getIndex().exists());

// Close the Redis connection
jedis.close();
System.out.println("Redis connection closed");

Router deleted
Index exists: false
Redis connection closed
